In [9]:
import numpy as np
import lime
import sklearn
import sklearn.ensemble
from anchor import utils
from anchor import anchor_tabular
import numpy as np
np.random.seed(1)
%load_ext autoreload
%autoreload 2

In [2]:
dataset_folder = '/home/kevin/iml-ws21-projects-risingnumpygods/datasets/'
dataset = utils.load_dataset('adult', balance=True, dataset_folder=dataset_folder, discretize=True)

Train 0.9350338780390594
Test 0.8489483747609943


In [4]:
explainer = anchor_tabular.AnchorTabularExplainer(
    dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)

In [5]:
idx = 0
np.random.seed(1)
print('Prediction: ', explainer.class_names[c.predict(dataset.test[idx].reshape(1, -1))[0]])
exp = explainer.explain_instance(dataset.test[idx], c.predict, threshold=0.95)

Prediction:  b'>50K'


In [6]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Anchor: Education = Bachelors AND Relationship = Husband AND Occupation = Sales
Precision: 0.97
Coverage: 0.02


In [6]:
import numpy as np

data = np.genfromtxt('../datasets/csv.txt', delimiter=',')
y_train = data[:, -1]
X_train = data[:, :-1]


(891, 10)

In [12]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Anchor.anchor import Anchor, Tasktype

c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5)
c.fit(X_train, y_train)
print('Train', sklearn.metrics.accuracy_score(y_train, c.predict(X_train)))

explainer = Anchor(Tasktype.TABULAR)


anchor = explainer.explain_instance(X_train[759].reshape(1, -1), c.predict, "smac", X_train, 100)

Train 1.0


ValueError: Expected 2D array, got 1D array instead:
array=[  1. 759.   1.   3.   1.   0. 230.   3.   1.   0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
anchor 

AnchorCandidate(_feature_mask=[3, 2, 4, 5, 6, 1, 0], _precision=1.0, _n_samples=200, _positive_samples=200, _coverage=0.015)